# Connect-K Game using Min-Max Algorithm with Alpha-Beta Pruning
- The Board class is the data structure that holds the Connect-'K' boards and the game operations
- The Connect-K board is 'w' cells wide and 'h' cells tall in height
- The underlying data structure is a 2-dimensional list or array
- The first dimension is the column; the second dimension is the row
- Every cell in the above list contains either a 0 or a 1. Player 1 is represented by 0 tiles, and Player

# 

In [1]:
#static class variables - shared across all instances
height = width = 0
K = int(input("Enter the winning condition for no. of consequecutive adjacent elements to be in sequence (horizontally/vertically/diagonally) :"))

while True:
    player_0_char = input("Enter character to identify player-0 in Game :")
    player_1_char = input("Enter character to identify player-1 in Game :")
    if player_0_char!=player_1_char :
        break
    else:
        print("Please enter a singular character, unqiue and distinct to identify the 2 players clearly !!!")

while max(height,width) < K :
    print(f"\n\t Kindly ensure that either of the dimensions (height/width) have value of at least '{K}' !!!")
    height = int(input("Enter Height of the Connect-K Game Board :"))
    width = int(input("Enter Width of the Connect-K Game Board :"))

Enter the winning condition for no. of consequecutive adjacent elements to be in sequence (horizontally/vertically/diagonally) : 4
Enter character to identify player-0 in Game : 🙋🏻
Enter character to identify player-1 in Game : 💻



	 Kindly ensure that either of the dimensions (height/width) have value of at least '4' !!!


Enter Height of the Connect-K Game Board : 7
Enter Width of the Connect-K Game Board : 6


In [2]:
def consecutiveElementsEqual(Arr, idx1, idx2, freqCount):
    
    def get_value(arr, indices):
        try:
            current = arr
            for idx in indices:
                current = current[idx]
            return current
        except (IndexError, TypeError):
            return None
    
    def get_direction_vector(start, end):
        direction = []
        for d in range(len(start)):
            diff = end[d] - start[d]
            if diff == 0:
                direction.append(0)
            else:
                direction.append(diff // abs(diff))
        return direction
    
    def get_points_along_line(start, end):
        if start == end:
            return [start]
        
        direction = get_direction_vector(start, end)
        points = []
        current = list(start)
        
        # Calculate number of steps needed
        steps = max(abs(end[d] - start[d]) for d in range(len(start)))
        
        for step in range(steps + 1):
            points.append(tuple(current))
            # Move to next point in direction
            current = [current[d] + direction[d] for d in range(len(start))]
        
        return points
    
    # Get all points along the line from idx1 to idx2
    line_points = get_points_along_line(idx1, idx2)
    
    # Get values along the line
    line_values = [get_value(Arr, point) for point in line_points]
    
    # Check if any values are None (out of bounds)
    if any(val is None for val in line_values):
        return False
    
    # Check for exactly 'freqCount' consecutive equal elements
    current_count = max_consecutive = 1
    
    for i in range(1, len(line_values)):
        if line_values[i] == line_values[i-1]:
            current_count += 1
            max_consecutive = max(max_consecutive, current_count)
        else:
            current_count = 1
    
    return max_consecutive == freqCount

<hr/>

## Board Environment Parameters

In [3]:
class Board(object):

    def __init__(self, orig=None, hashValue=None):

        self.HEIGHT = height
        self.WIDTH = width
        
        if(orig):
            self.board = [list(col) for col in orig.board]
            self.numMoves = orig.numMoves
            self.lastMove = orig.lastMove
            return

        # creates from hashValue - NOTE: Does not understand move order
        elif(hashValue):
            self.board = []
            self.numMoves = 0
            self.lastMove = None

            # convert to base-K
            digits = []
            while hashValue:
                digits.append(int(hashValue % (K-1)))
                hashValue //= (K-1)

            col = []
            
            for item in digits:
                # 2 indicates new column
                if item == 2:
                    self.board.append(col)
                    col = []
                
                # otherwise directly append base number
                else:
                    col.append(item)
                    self.numMoves += 1
            return

        # create new
        else:
            self.board = [[] for x in range(self.WIDTH)]
            self.numMoves = 0
            self.lastMove = None
            return


        # Mutations

    # Puts a piece in the appropriate column and checks to see if it was a winning move.
    # Pieces are either 1 or 0 and automatically decided by algorithm
    def makeMove(self, column):
        # update board data
        piece = self.numMoves % 2
        self.lastMove = (piece, column)
        self.numMoves += 1
        self.board[column].append(piece)


    # Observations

    # Generates a list of the valid children of the board
    # A child is of the form (move_to_make_child, child_object)
    def children(self):
        children = []
        for i in range(width):
            if len(self.board[i]) < height:
                child = Board(self)
                child.makeMove(i)
                children.append((i, child))
        return children

    # Returns:
    def isTerminal(self):
        for i in range(0,self.WIDTH):
            for j in range(0,self.HEIGHT):
                # Vertical , Horizontal , Semi-Diagonals, Anti-Diagonals
                if consecutiveElementsEqual(self.board, [i,j], [i+(K-1),j], K) or consecutiveElementsEqual(self.board, [i,j], [i,j+(K-1)], K) or not j + (K-1) > self.HEIGHT and consecutiveElementsEqual(self.board, [i,j], [i+(K-1),j+(K-1)], K) or not j - (K-1) < 0 and consecutiveElementsEqual(self.board, [i,j], [i+(K-1),j-(K-1)], K):
                    return [self.board[i][j]], True
                    
        if self.isFull():
            return [0,1],True
            
        return [],False

    # Returns a unique decimal number for each board object based on the
    # underlying data
    def hashValue(self):

        power = 0
        hashValue = 0

        for column in self.board:

            # add 0 or 1 depending on piece
            for piece in column:
                hashValue += piece * ((K-1) ** power)
                power += 1

            # add a 2 to indicate end of column
            hashValue += 2 * ((K-1) ** power)
            power += 1

        return hashValue

        # Utilities

    # Return true iff the game is full
    def isFull(self):
        return self.numMoves == height*width

    # Prints out a visual representation of the board
    # X's are 1's and 0's are 0s
    def display(self):
        print()
        
        # Determine if we're using emojis (multi-character symbols)
        is_emoji = len(player_0_char.encode('utf-8'))>1 or len(player_1_char.encode('utf-8'))>1 
        
        print()
        print('+'+'-'*6+"-" *2* self.WIDTH+'+' if is_emoji else '+'+'-'*8+"-" *2* self.WIDTH+'+')
        for rowNum in range(self.HEIGHT - 1, -1, -1):
            row = "|"
            for colNum in range(self.WIDTH):
                if len(self.board[colNum]) > rowNum:
                    row += (" " if not is_emoji else '') + (player_1_char if self.board[colNum][rowNum] else player_0_char) + (" " if not is_emoji else '')+"|"
                else:
                    row += (" "*3 if not is_emoji else ' '*2)+"|"
            print(row)
            print('+'+'-'*6+"-" *2* self.WIDTH+'+' if is_emoji else '+'+'-'*8+"-" *2* self.WIDTH+'+')
        print("Previously, player:", self.lastMove[0], ", moved into column-", self.lastMove[1]+1, "with '", (player_1_char if self.lastMove[0] else player_0_char), "', above seen.")
        print("Move count =", self.numMoves)

<hr/>

## Player types inheriting from a central class

In [4]:
class Player:

    def __init__(self, iterationsDepthLimit, isInitialStartPlayer):

        self.isInitialStartPlayer = isInitialStartPlayer
        self.iterationsDepthLimit = iterationsDepthLimit

    # Returns a heuristic for the board position
    # Good positions for 0 pieces are positive and good positions for 1 pieces are be negative
    def heuristic(self, board):
        cumul_path_cost = 0
        state = board.board
        for i in range(0, board.WIDTH):
            for j in range(0, board.HEIGHT):
                
                ''' check horizontal streaks '''
                # add player one streak scores to cumul_path_cost
                for d in range(K):
                    if consecutiveElementsEqual(state, [i,j], [i+d,j], d+1) and state[i][j]==0:
                        cumul_path_cost += d*10**(2*d)

                # subtract player two streak score to cumul_path_cost
                for d in range(K):
                    if consecutiveElementsEqual(state, [i,j], [i+d,j], d+1) and state[i][j]==1:
                        cumul_path_cost -= d*10**(2*d)

                ''' check vertical streaks '''
                # add player one vertical streaks to cumul_path_cost
                for d in range(K):
                    if consecutiveElementsEqual(state, [i,j], [i,j+d], d+1) and state[i][j]==0:
                        cumul_path_cost += d*10**(2*d)

                # subtract player two streaks from cumul_path_cost
                for d in range(K):
                    if consecutiveElementsEqual(state, [i,j], [i,j+d], d+1) and state[i][j]==1:
                        cumul_path_cost -= d*10**(2*d)

                
                ''' check positive semi-diagonal streaks '''
                # add player one streaks to cumul_path_cost
                for d in range(K):
                    if not j + (K-1) > board.HEIGHT and consecutiveElementsEqual(state, [i,j], [i+d,j+d], d+1) and state[i][j]==0:
                        cumul_path_cost += d*10**(2*d)

                # add player two streaks to cumul_path_cost
                for d in range(K):
                    if not j + (K-1) > board.HEIGHT and consecutiveElementsEqual(state, [i,j], [i+d,j+d], d+1) and state[i][j]==1:
                        cumul_path_cost -= d*10**(2*d)

                
                ''' check negative anti-diagonal streaks '''
                # add  player one streaks
                for d in range(K):
                    if not j - (K-1) < 0 and consecutiveElementsEqual(state, [i,j], [i+d,j-d], d+1) and state[i][j]==0:
                        cumul_path_cost += d*10**(2*d)

                # subtract player two streaks
                for d in range(K):
                    if not j - (K-1) < 0 and consecutiveElementsEqual(state, [i,j], [i+d,j-d], d+1) and state[i][j]==1:
                        cumul_path_cost -= d*10**(2*d)
                        
        return cumul_path_cost

In [5]:
# Min-Max Algorithm Player
class PlayerMM(Player):
    def __init__(self, iterationsDepthLimit, isInitialStartPlayer):
        super().__init__(iterationsDepthLimit, isInitialStartPlayer)

    # returns the optimal column to move in by implementing the MiniMax algorithm
    def findMove(self, board):
        #return self.mmH(board, self.iterationsDepthLimit, self.isInitialStartPlayer)
        score, move = self.miniMax(board, self.iterationsDepthLimit, self.isInitialStartPlayer)
        print(f"Player-{int(self.isInitialStartPlayer)}", ", moved into column-", int(move)+1,"with'",(player_1_char if self.isInitialStartPlayer else player_0_char),"' below seen.")
        return move

    # findMove helper function using miniMax algorithm
    def miniMax(self, board, iters_depth, player):
        if len(board.isTerminal()[0])>1:
            return float('-inf') if player else float('+inf'), -1
        elif iters_depth == 0:
            return self.heuristic(board), -1

        if player:
            bestScore = float('-inf')
            shouldReplace = lambda x: x > bestScore
        else:
            bestScore = float('+inf')
            shouldReplace = lambda x: x < bestScore

        bestMove = -1

        children = board.children()
        for child in children:
            move, childboard = child
            temp = self.miniMax(childboard, iters_depth-1, not player)[0]
            if shouldReplace(temp):
                bestScore = temp
                bestMove = move
        return bestScore, bestMove

    # minimax helper function
    def mmH(self, board, iters_depth, player):

        if iters_depth == 0:
            boards = board.children()
            scores = {}
            for i in boards:
                scores[i[0]] = self.heuristic(i[1])
            if player:
                return max(scores, key=lambda k: scores[k])
            else:
                return min(scores, key=lambda k: scores[k])
        else:
            return self.mmH(board,iters_depth-1,not player)

In [6]:
# Alpha-Beta Pruning algorithm Player
class PlayerAB(Player):

    def __init__(self, iterationsDepthLimit, isInitialStartPlayer):
        super().__init__(iterationsDepthLimit, isInitialStartPlayer)

    # returns the optimal column to move in by implementing the Alpha-Beta algorithm
    def findMove(self, board):
        score, move = self.alphaBeta(board, self.iterationsDepthLimit, self.isInitialStartPlayer, float('-inf'), float('+inf'))
        return move

    # findMove helper function, utilizing alpha-beta pruning within the  minimax algorithm
    def alphaBeta(self, board, iters_depth, player, alpha, beta):
        if len(board.isTerminal()[0])>1 :
            return float('-inf') if player else float('+inf'), -1
        
        elif iters_depth == 0:
            return self.heuristic(board), -1

        if player:
            bestScore = float('-inf')
            shouldReplace = lambda x: x > bestScore
        else:
            bestScore = float('+inf')
            shouldReplace = lambda x: x < bestScore

        bestMove = -1

        children = board.children()
        for child in children:
            move, childboard = child
            temp = self.alphaBeta(childboard, iters_depth-1, not player, alpha, beta)[0]
            if shouldReplace(temp):
                bestScore = temp
                bestMove = move
            if player:
                alpha = max(alpha, temp)
            else:
                beta = min(beta, temp)
            if alpha >= beta:
                break
        return bestScore, bestMove

In [7]:
class ManualPlayer(Player):
    
    def findMove(self, board):
        opts = " "
        for c in range(board.WIDTH):
            opts += " " + (str(c + 1) if len(board.board[c]) < height else ' ') + "  "
        print("\nAvailable columns options :",opts)
        col = input("Place an " + (player_0_char if self.isInitialStartPlayer else player_1_char) + " in column: ")

        while col not in opts:
            print("Column is already stacked up and full !!!\n Use some other columns amongst the available options")
            print("\nAvailable columns options :",opts)
            col = input("Place an " + (player_0_char if self.isInitialStartPlayer else player_1_char) + " in column: ")
        
        return int(col) - 1

<hr/>

## Game Play Run/Execution

In [8]:
class Game:

    def __init__(self, startBoard, player1, player2):
        self.startBoard = startBoard
        self.player1 = player1
        self.player2 = player2


    #  Simulating a Game for Connect-K using the Min-Max Algo with alpha-beta pruning
    def simulateLocalGame(self):

        board = Board(orig=self.startBoard)
        isPlayer1 = True

        while(True):

            #finds the move to make
            if isPlayer1:
                move = self.player1.findMove(board)
            else:
                move = self.player2.findMove(board)

            #makes the move
            board.makeMove(move)
            board.display()

            #determines if the game is over or not
            winner, isOver = board.isTerminal()
            if isOver and len(winner)>1:
                print("It is a draw!")
                break
            elif isOver and len(winner)==1 and winner[0]==0:
                print("Player 0 wins!")
                break
            elif isOver and len(winner)==1 and winner[0]==0:
                print("Player 1 wins!")
                break
            else:
                isPlayer1 = not isPlayer1


In [ ]:
playerOneSetterFlag = True
print("\nSelect the type of Players as per below listed options (Default value: 'C', i.e, Manual Player) :-\n\tA. Min-Max algorithm based A.I Agent\n\tB. Alpha-Beta pruning algorithm based A.I Agent\n\tC. Manual Player (Human user)\n")

print()
print(f"Player-0 (plays-'{player_0_char}') :-")
agentPlayerType_0 = input("Select Agent/Player Type from available option (A-C) :")
repetitionThreshold_0 = int(input("Enter iteration depth limit bound :"))
agentPlayerFunction_0 = PlayerMM if agentPlayerType_0.upper() == 'A' else (PlayerAB if agentPlayerType_0.upper() == 'B' else ManualPlayer)

print()
print(f"Player-1 (plays-'{player_1_char}'):-")
agentPlayerType_1 = input("Select Agent/Player Type from available option (A-C) :")
repetitionThreshold_1 = int(input("Enter iteration depth limit bound :"))
agentPlayerFunction_1 = PlayerMM if agentPlayerType_1.upper() == 'A' else (PlayerAB if agentPlayerType_1.upper() == 'B' else ManualPlayer)

print("_"*100)
if __name__ == "__main__":
    game = Game(Board(), agentPlayerFunction_0(repetitionThreshold_0, playerOneSetterFlag), agentPlayerFunction_1(repetitionThreshold_1, not playerOneSetterFlag))
    game.simulateLocalGame()


Select the type of Players as per below listed options (Default value: 'C', i.e, Manual Player) :-
	A. Min-Max algorithm based A.I Agent
	B. Alpha-Beta pruning algorithm based A.I Agent
	C. Manual Player (Human user)


Player-0 (plays-'🙋🏻') :-


Select Agent/Player Type from available option (A-C) : C
Enter iteration depth limit bound : 3



Player-1 (plays-'💻'):-


Select Agent/Player Type from available option (A-C) : A
Enter iteration depth limit bound : 5


____________________________________________________________________________________________________

Available columns options :   1   2   3   4   5   6  


...



+------------------+
|  |  |  |  |  |  |
+------------------+
|  |  |  |  |  |  |
+------------------+
|  |  |  |  |  |  |
+------------------+
|  |  |  |  |  |  |
+------------------+
|  |💻|  |  |  |  |
+------------------+
|💻|🙋🏻|  |  |  |  |
+------------------+
|💻|🙋🏻|  |🙋🏻|  |🙋🏻|
+------------------+
Previously, player: 0 , moved into column- 4 with ' 🙋🏻 ', above seen.
Move count = 7
Player-0 , moved into column- 1 with' 🙋🏻 ' below seen.


+------------------+
|  |  |  |  |  |  |
+------------------+
|  |  |  |  |  |  |
+------------------+
|  |  |  |  |  |  |
+------------------+
|  |  |  |  |  |  |
+------------------+
|💻|💻|  |  |  |  |
+------------------+
|💻|🙋🏻|  |  |  |  |
+------------------+
|💻|🙋🏻|  |🙋🏻|  |🙋🏻|
+------------------+
Previously, player: 1 , moved into column- 1 with ' 💻 ', above seen.
Move count = 8

Available columns options :   1   2   3   4   5   6  


Place an 💻 in column:  5




+------------------+
|  |  |  |  |  |  |
+------------------+
|  |  |  |  |  |  |
+------------------+
|  |  |  |  |  |  |
+------------------+
|  |  |  |  |  |  |
+------------------+
|💻|💻|  |  |  |  |
+------------------+
|💻|🙋🏻|  |  |  |  |
+------------------+
|💻|🙋🏻|  |🙋🏻|🙋🏻|🙋🏻|
+------------------+
Previously, player: 0 , moved into column- 5 with ' 🙋🏻 ', above seen.
Move count = 9
Player-0 , moved into column- 1 with' 🙋🏻 ' below seen.


+------------------+
|  |  |  |  |  |  |
+------------------+
|  |  |  |  |  |  |
+------------------+
|  |  |  |  |  |  |
+------------------+
|💻|  |  |  |  |  |
+------------------+
|💻|💻|  |  |  |  |
+------------------+
|💻|🙋🏻|  |  |  |  |
+------------------+
|💻|🙋🏻|  |🙋🏻|🙋🏻|🙋🏻|
+------------------+
Previously, player: 1 , moved into column- 1 with ' 💻 ', above seen.
Move count = 10

Available columns options :   1   2   3   4   5   6  


Place an 💻 in column:  3




+------------------+
|  |  |  |  |  |  |
+------------------+
|  |  |  |  |  |  |
+------------------+
|  |  |  |  |  |  |
+------------------+
|💻|  |  |  |  |  |
+------------------+
|💻|💻|  |  |  |  |
+------------------+
|💻|🙋🏻|  |  |  |  |
+------------------+
|💻|🙋🏻|🙋🏻|🙋🏻|🙋🏻|🙋🏻|
+------------------+
Previously, player: 0 , moved into column- 3 with ' 🙋🏻 ', above seen.
Move count = 11


In [ ]:
print('END')